# Steal The Files - Dokumentation (DE)
Client | Server | Core - JavaFX 2D-Plattformer

**Inhalt**
1. &Uuml;berblick &amp; Architektur
2. Setup &amp; Start
3. Spiellogik: Schritte, Entscheidungen, Notizen
4. GUI/Navi: Schritte, Entscheidungen, Notizen
5. Levelaufbau: Schritte, Entscheidungen, Notizen
6. Audio &amp; Settings
7. Netzwerk &amp; Daten
8. Debugging &amp; offene Punkte
9. Roadmap-Nuggets

## 1. &Uuml;berblick &amp; Architektur
- Ziel laut Code: USB-Stick holen -> Files freischalten -> Folder einsammeln -> Gefahren &uuml;berleben -> Finish ber&uuml;hren.
- Module: **core** (Entities, Events, Packets/Serialization), **client** (JavaFX-Screens, Level, Audio, Input), **server** (TCP-Listener, Packet-Routing, DB-Hooks vorbereitet).
- Startpfad: `Game.start` l&auml;dt `config.json`, setzt Sound muted/volume, registriert Listener, baut `ScreenManager`, erzeugt MainMenu/Settings, w&auml;hlt Level per `currentLevel` (Tutorial/Second/Boss).
- Game-Loop: eigener Thread im `ScreenManager` (~200 Hz) berechnet `delta`+FPS, ruft `currentScreen.update(delta)` &uuml;ber `Platform.runLater`.
- Pipeline zur Laufzeit:
```text
Scene-Input -> InputManager -> ScreenManager.update(delta) -> GameScreen.update(delta)
  -> Physik/Kollision -> Kamera-Offset -> HUD-Update -> Render/Sound
```

## 2. Setup &amp; Start
- Anforderungen: JDK 19, JavaFX 17, Maven; Assets unter `/assets`.
- Build (alle Module):
```bash
mvn clean install
```
- Start aus IDE: Server `de.cyzetlc.hsbi.Server` (TCP 25570), Client `de.cyzetlc.hsbi.game.Game` (verbindet zu localhost, zeigt Loading -> MainMenu).
- Wichtige Config-Felder: `currentLevel`, `soundVolume` (0-1), `soundMuted` (bool), `messages.game.prefix`, `mysql`-Block (nicht auto-aktiv im Client), `tutorialFinished`.

## 3. Spiellogik: Schritte, Entscheidungen, Notizen
**Schritt-f&uuml;r-Schritt (aus GameScreen + Entities/Blocks)**
- Input erfassen: `InputManager` tracked gedr&uuml;ckte Tasten + `pollJustPressed` f&uuml;r Toggles.
- Player init: `EntityPlayer` (maxHealth=3) reset bei Screen-Wechsel, NameTag & Sprite gesetzt.
- Bewegungsparameter: `gravity=15`, `moveSpeed=450`, `jumpPower=800` (Jump/Speed-Perks skalieren +25% f&uuml;r 10s).
- Update-Loop: berechnet dx/dy, addiert Gravitation, bildet Next-Bounds.
- Kollisionen: zuerst Platforms (Landung/Seiten), dann Blocks (Items/Perks/Hazards/Enemy); `pendingBlocks` f&uuml;r frisch gespawnte Laser.
- Interaktion: Taste E triggert `GasBarrierBlock.deactivate()` nur mit Flipper-Flag.
- Gegner: `RobotEnemyBlock` patrouilliert innerhalb [minX,maxX], dreht Richtung zum Spieler, feuert `LaserBlock` bei |dx|<440, Cooldown 1.2s, Laser 4s Lebensdauer, 1 Schaden (verschwindet auch in GodMode). Stomp von oben killt.
- Hazards: `LavaBlock` -0.25 HP/Tick (kein Schaden in GodMode).
- Items/Progress: `USBStickBlock` setzt `canCollectFiles=true`; `FolderBlock` z&auml;hlt nur dann und deaktiviert sich; `FlipperItem` setzt Schl&uuml;ssel-Flag f&uuml;r Schranke.
- Kamera: Dead-Zone `marginX=400`, `marginY=150`, smoothing `cameraSmooth=0.1`; alle Sprites offsetten zur Kamera.
- HUD: Quest-Text, Files `X/Y` (aktive Folder), Flipper-Hinweis, Health-Herzen (maxHealth=3), Debug-Bar bei F2, Tooltips bei F1.
- Game States: Game Over bei HP<=0 oder Fall -> MainMenu; Pause blendet Overlay ein (Logik l&auml;uft weiter).

**Design-Entscheidungen (im Code sichtbar)**
- Delta-basierte Physik im eigenen Thread, Render auf JavaFX-Thread: vermeidet UI-Blockaden.
- GodMode/NoClip Toggle (F3) f&uuml;r schnelles Debugging; GodMode verhindert HP-Reduktion, NoClip &uuml;berspringt Kollisionen.
- Laser-Lifetime fest (4s) statt Distanz-basiert, um Berechnung schlank zu halten.
- Folder erst nach USB unlockbar -> klares Progress-Gate.
- Hard Limits f&uuml;r Kamera-Smoothing und Dead-Zone, damit Bewegungen nicht ruckeln.

**Technische Notizen / Probleme**
- Pause stoppt die Logik nicht; nur Overlay wird sichtbar.
- `Level.update()` ist leer; Moving Blocks laufen &uuml;ber Block-Update/Entity-Update.
- LevelFinished-Statistiken sind Hardcoded (Zeit/Score/Kills/HP-Loss).
- Kamera-Clamps pro Level fehlen (kann Offscreen scrollen).

## 4. GUI/Navi: Schritte, Entscheidungen, Notizen
**Schritt-f&uuml;r-Schritt Aufbau**
- LoadingScreen: animierter Hintergrund, Fortschrittsbalken (Timeline 3s), l&auml;dt `Material`-Textures in Thread, wechselt dann ins MainMenu.
- MainMenuScreen: Parallax-Background (zwei Images, Slide), Buttons Start/Multiplayer/Einstellungen/Beenden, Dummy-Achievements, Footer mit Version/Copyright.
- SettingsScreen: Rechts oben Panel (220x90), Slider 0-1, Mute-Button; persistiert direkt in `config.json`, Slider deaktiviert wenn muted.
- MultiplayerScreen: Drei statische Serverkarten (Slot/MOTD), Zur&uuml;ck-Button.
- GameScreen: Root clear, Hintergrund animiert, Spieler zeichnen, Level (Platforms/Blocks) laden, Buttons Zur&uuml;ck/Pause, HUD (Quest, Files, Debug optional, Health-Bar), Pause-Overlay.
- LevelFinishedScreen: Halbtransparentes Overlay, Stats-Text (Zeit/Score/Kills/Verlorene Leben), Buttons f&uuml;r n&auml;chstes Level (falls vorhanden) oder Hauptmen&uuml;.

**Design-Entscheidungen**
- Fenster Fullscreen, Stage undecorated, Fullscreen-Exit disabled -> immersiv, kein OS-Frame.
- Buttons/HUD oben links; Health rechts oben -> Sichtfeld f&uuml;r Spielfigur bleibt frei.
- Dummy-Achievements/Serverliste als Platzhalter, aber UI-Struktur steht.
- Pause bewusst nur Overlay, um Lauf-Logik weiter zu beobachten (Debug-Fokus).

**Technische Notizen / Probleme**
- Pause-Overlay hat keinen Logik-Stopp.
- LevelFinished zeigt Platzhalter-Stats, nicht die echten Werte.
- Stage auf fixe Fullscreen-Gr&ouml;&szlig;e, kein Resize-Handling.
- Kein eigener UI-Thread; alle UI-Updates &uuml;ber JavaFX-Thread per `Platform.runLater`.

## 5. Levelaufbau: Schritte, Entscheidungen, Notizen
**TutorialLevel (draw)**
- Plattformen staffeln Boden/H&uuml;gel; Lava zwischen Gaps (per `placeLavaBetweenPlatforms`).
- Blocks: USB fr&uuml;h, JumpBoost, SpeedBoost, ServerBlock, FolderBlock, FloatingPlatform verbindet Inseln, Finish rechts.
- `onFinish`: setzt `tutorialFinished=true`, speichert Config.
- Next-Level: `SecondLevel`.

**SecondLevel (draw)**
- Viele Plattformen inkl. schwebender, 8 FolderBlocks, FlipperItem nahe Start, JumpBoost mittig, RobotEnemy auf Endplattform, Finish ganz rechts; GasBarrier vorbereitet (auskommentiert).
- Einmaliger Spawn-Sound beim Betreten.
- Next-Level: `BossLevel`.

**BossLevel (draw)**
- Breite Startplattform, vier RobotEnemyBlocks in Reihe, Finish dahinter; kein Next-Level.

**Design-Entscheidungen**
- USB zuerst, dann Folder -> Progress-Gate erzwingt Route.
- FlipperItem im SecondLevel fr&uuml;h -> GasBarrier-Mechanik sofort verf&uuml;gbar, falls eingebaut.
- FloatingPlatform + Moving Enemy f&uuml;r Abwechslung ohne komplexe Physik.

**Technische Notizen / Probleme**
- Level-Koordinaten hart kodiert; kein Editor/JSON.
- `Level.update()` leer -> keine levelweiten Ticks (z.B. globale Timer).
- Kamera hat keine Bounds pro Level.
- GasBarrier im SecondLevel aktuell auskommentiert.


## 6. Audio &amp; Einstellungen
- SoundManager: Media-Caching, globale Lautst&auml;rke (0-1), Mute-Flag, Boost via dB-Faktor, Background-Musik `playBackground(Music.MENU, true)`, Ducking f&uuml;r Effekte (`playWithDuck`).
- SettingsScreen koppelt Slider/Mute direkt an SoundManager und speichert sofort in Config (volume/muted).
- Effekte im Code: Klicks bei Sammeln/Interaktion, Buff-Sounds (Jump/Speed), USB, Laser/Robot; Tutorial pr&uuml;ft `tutorialFinished`, um Sounds ggf. zu drosseln.
- Assets werden im LoadingScreen vorgew&auml;rmt (Material-Textures) -> weniger Ruckler beim ersten Draw.

## 7. Netzwerk &amp; Daten
- Client: `connectAndRun` -> TCP localhost:25570, sendet `ClientLoginPacket` (UUID), startet Receiver/Sender.
- Sender: alle 50ms `ClientDataPacket` (Heartbeat).
- Server: akzeptiert Sockets, legt `MultiClientHandler` an, sendet periodisch `UserMessagePacket("Connection is stable")`, verarbeitet Pakete via `ReceivePacketEvent` (cancelable).
- Packet-Implementierungen und `SerializationUtils` liegen im Modul **core**.
- DB: `QueryHandler`/`MySQLCredentials` vorhanden, aber Verbindungsaufbau im Server deaktiviert (Placeholder).
- Score-Senden: `Client.sendFinalScore(long)` mit `ClientSubmitScorePacket` (Level-Name hart "Level_1").

## 8. Debugging &amp; offene Punkte
- Live-Toggles: F1/F2/F3, Pause-Overlay (ohne Logik-Stopp), Health-Bar reagiert auf Schaden/Heilung, Files-Z&auml;hler erst nach USB.
- Manuelle Tests aus Code: USB aufnehmen -> Folder einsammeln -> Finish triggert LevelFinished; Jump/Speed-Buffs laufen 10s, resetten; RobotEnemy stompbar, Laser nach 4s weg; Flipper + GasBarrier (falls aktiv) &uuml;ber E deaktivierbar.
- Offene Baustellen: echte Pause (Update-Schleife stoppen), `Level.update()` nutzen f&uuml;r mover/Timer, LevelFinished-Stats mit echten Werten, Kamera-Clamps pro Level, GasBarrier im SecondLevel aktivieren, Multiplayer/Highscore-Flow mit echter DB.


## 9. Roadmap-Nuggets
- Kurzfristig: Pause wirklich anhalten, LevelFinished mit echten Stats, Kamera-Bounds je Level, GasBarrier-Route im SecondLevel einschalten.
- Mittelfristig: Level.update f&uuml;r globale Ticks/Mover, Konfigurierbare Level aus JSON/Editor, HUD um Timer/Score erweitern.
- Langfristig: Multiplayer/Highscore mit DB finalisieren, Achievements/Serverliste aus Backend speisen.
